# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#| export
from dataclasses import dataclass

import sympy as smp
from exex.utils import *

In [ ]:
#| export
class Event(list):
    def __call__(self, *args, **kwargs):
        for item in self:
            item(*args, **kwargs)

### Unit

In [ ]:
#| export
@dataclass
class Unit:
    """
    Default Units
    """
    
    # SI Unit
    LENGTH = 'meter'
    MASS = 'kilogram'
    TIME = 'second'
    TEMPERATURE = 'kelvin'
    
    # Derived from SI Unit
    MOLE = 'mole'
    SPECIFIC_HEAT = 'joule / (kilogram kelvin)'
    PRESSURE = 'pascal'

### Data

In [ ]:
#| export
class Object:
    pass

In [ ]:
#| export
class PropertyData(dict):
    pass

In [ ]:
c = PropertyData({'0': {'mass': 2, 'object': Object()}})

In [ ]:
c['0']

{'mass': 2, 'object': <__main__.Object>}

In [ ]:
c['1'] = {'mass': 2.1, 'object': 'XXX'}

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(c)

In [ ]:
df

,0,1
mass,2,2.1
object,<__main__.Object object at 0x7f8c21a74ee0>,XXX


### Proprety

In [ ]:
#| export
class PropertyObservable:
    def __init__(self, compound):
        self._data = PropertyData()
        self.compound = compound
        self._connections = []
        self.laws = dict()
        self.func_changed = Event()
        
        # print(f'self.compound={self.compound}')
        # print(self.compound.__dict__)
        # print(f'formula={self.compound.formula}')
        
        self.symbol = smp.symbols(f'{self.abbreviate}', real=True)
    
    def add_data(self, time, value):
        self._data[time] = {'value': value}
    
    def add_law(self, law):
        if not law in self.laws:
            self.laws[camel_to_snake(law.__class__.__name__)] = law

In [ ]:
#| export
class Mass(PropertyObservable):
    def __init__(self, compound):
        self.abbreviate = 'm'
        super().__init__(compound)

In [ ]:
#| export
class Mole(PropertyObservable):
    def __init__(self, compound):
        self.abbreviate = 'n'
        super().__init__(compound)

In [ ]:
#| export
class Pressure(PropertyObservable):
    def __init__(self, compound):
        self.abbreviate = 'P'
        super().__init__(compound)

In [ ]:
#| export
class Volume(PropertyObservable):
    def __init__(self, compound):
        self.abbreviate = 'V'
        super().__init__(compound)

In [ ]:
#| export
class Temperature(PropertyObservable):
    def __init__(self, compound):
        self.abbreviate = 'T'
        super().__init__(compound)

### Law

In [ ]:
#| export
class Law:
    def _run_config(self):
        self._config_properties()
    
    def _config_properties(self):
#         for p in self.properties:
#             name = camel_to_snake(p.__name__)
            
#             if not name in self.compound.properties:
#                 self.compound.properties[name] = p(compound=self.compound)
            
#             self.compound.properties[name].add_law(self)
        
        for p in self.x_properties:
            name = camel_to_snake(p['object'].__name__)
            
            if not name in self.compound.properties:
                self.compound.properties[name] = p['object'](compound=self.compound)
            
            self.compound.properties[name].add_law(self)

### Mediator

In [ ]:
#| export
class System:
    def __init__(self):
        self.reactions = dict()